In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import pandas as pd
%load_ext autoreload
%autoreload 2

data_path = "/data/desponds/data/Summarization/"

In [ ]:
from preprocessing import preprocessing_summarization
datasets, tokenized = preprocessing_summarization()

In [ ]:
import pickle
with open('/data/desponds/data/Summarization/tokenized.pickle', 'wb') as handle:
    pickle.dump(tokenized, handle)
with open('/data/desponds/data/Summarization/datasets.pickle', 'wb') as handle:
    pickle.dump(datasets, handle)

In [ ]:
import pickle
with open('/data/desponds/data/Summarization/tokenized.pickle', 'rb') as handle:
    tokenized = pickle.load(handle)
with open('/data/desponds/data/Summarization/datasets.pickle', 'rb') as handle:
    datasets = pickle.load(handle)

In [ ]:
from training import get_trainers_summarization
trainers = get_trainers_summarization(data_path, tokenized)

In [ ]:
trainers['fr'].train()
trainers['en'].train()

## Translation

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import pandas as pd
%load_ext autoreload
%autoreload 2

data_path = "/data/desponds/data/Summarization/"

In [ ]:
import pickle
with open('/data/desponds/data/Summarization/tokenized.pickle', 'rb') as handle:
    tokenized = pickle.load(handle)
with open('/data/desponds/data/Summarization/datasets.pickle', 'rb') as handle:
    datasets = pickle.load(handle)

In [ ]:
d = datasets['fr']['test'][0]['source'][:1900]
d

In [ ]:
from translation import translate_fr_en_1,translate_fr_en_2, translate_en_fr
print(translate_fr_en_1(datasets['fr']['test'][0]['source'])) #with truncate = True 
# translate_fr_en_2([d])

In [ ]:
print(translate_fr_en_1(datasets['fr']['test'][0]['source'][:1900]))

In [ ]:
from translation import translate_fr_en_summarization
translated_dataset = datasets['fr']['test'].map(translate_fr_en_summarization)
with open('/data/desponds/data/Summarization/translated_dataset.pickle', 'wb') as handle:
    pickle.dump(translated_dataset, handle)

In [ ]:
from translation import translate_fr_en
d = datasets['fr']['test'][0]['source']
translate_fr_en(d)

In [12]:
len(d)

458

## Inference

In [ ]:
from training import get_models_summarization
models = get_models_summarization(trainers)

In [ ]:
results = {}
results['fr'] = models['fr'].predict(tokenized['fr']['test'])
results['en'] = models['en'].predict(tokenized['en']['test'])
results

In [11]:
import pandas as pd
final_results = { k: v[2] for k,v in results.items()}
pd.DataFrame(final_results)
#{'fr': {'test_loss': 2.488802194595337,
#   'test_rouge1': 0.14,
#   'test_rouge2': 0.0482,
#   'test_rougeL': 0.1284,
#   'test_rougeLsum': 0.1284,
#   'test_gen_len': 18.5642,
#   'test_runtime': 727.3367,
#   'test_samples_per_second': 11.941,
#   'test_steps_per_second': 1.493},
#  'en': {'test_loss': 2.4726946353912354,
#   'test_rouge1': 0.2258,
#   'test_rouge2': 0.0747,
#   'test_rougeL': 0.1972,
#   'test_rougeLsum': 0.1972,
#   'test_gen_len': 18.3344,
#   'test_runtime': 1552.1017,
#   'test_samples_per_second': 12.308,
#   'test_steps_per_second': 1.539}}

,fr,en
test_loss,2.488802,2.472695
test_rouge1,0.140000,0.225800
test_rouge2,0.048200,0.074700
test_rougeL,0.128400,0.197200
test_rougeLsum,0.128400,0.197200
test_gen_len,18.564200,18.334400
test_runtime,727.336700,1552.101700
test_samples_per_second,11.941000,12.308000
test_steps_per_second,1.493000,1.539000


In [10]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/data/desponds/data/Summarization/trainer_fr/checkpoint-38000")
summarizer(text)

[{'summary_text': '<extra_id_0>. Vous avez raison. »'}]